In [8]:
!pip install --quiet opencv-python
!pip install --quiet pillow

In [3]:
import cv2
from pathlib import Path

def extract_frames(video_file: Path, output_dir: Path, interval: int,
                   prefix: str = 'frame', fmt: str = 'png') -> None:
    output_dir.mkdir(parents=True, exist_ok=True)
    cap = cv2.VideoCapture(str(video_file))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open {video_file}")
    frame_idx = save_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx % interval == 0:
            out_path = output_dir / f"{prefix}_{save_idx:04d}.{fmt}"
            cv2.imwrite(str(out_path), frame)
            save_idx += 1
        frame_idx += 1
    cap.release()
    print(f"Extracted {save_idx} frames to '{output_dir}'")

In [5]:
video = Path('/content/demo.mp4')
output = Path('/content/frames')

extract_frames(video, output, interval=30, prefix='frame', fmt='png')

Extracted 4 frames to '/content/frames'


In [7]:
from PIL import Image
import os

def image_to_c_array(path):
    img = Image.open(path).convert('L').resize((64, 64))
    data = list(img.getdata())
    with open('frame_array.h', 'w') as f:
        f.write('const uint8_t frame[64][64] = {\n')
        for i in range(64):
            f.write('  {' + ', '.join(str(data[i*64 + j]) for j in range(64)) + '},\n')
        f.write('};')

image_to_c_array('frames/frame_0002.png')

In [12]:
from pathlib import Path
from PIL import Image
import sys

def convert_image_to_c_array(input_path: Path,
                             output_path: Path,
                             width: int = 64,
                             height: int = 64,
                             var_name: str = 'frame'):
    if not input_path.is_file():
        raise FileNotFoundError(f"No file at {input_path}")
    img = Image.open(input_path).convert('L').resize((width, height))
    pixels = list(img.getdata())

    with open(output_path, 'w') as f:
        f.write('#ifndef FRAME_ARRAY_H\n#define FRAME_ARRAY_H\n\n')
        f.write('#include <stdint.h>\n\n')
        f.write(f'const uint8_t {var_name}[{height}][{width}] = {{\n')
        for row in range(height):
            row_vals = pixels[row*width:(row+1)*width]
            vals_str = ', '.join(str(v) for v in row_vals)
            f.write(f'    {{ {vals_str} }},\n')
        f.write('};\n\n#endif // FRAME_ARRAY_H\n')

    print(f"Wrote C header to {output_path}")

In [13]:
inp = Path('/content/frames/frame_0002.png')
out = Path('/content/frame_array.h')

convert_image_to_c_array(inp, out, width=64, height=64, var_name='frame')

Wrote C header to /content/frame_array.h
